In [2]:
from zope.interface import Interface, implementer
from zope.interface.verify import verifyObject
from abc import ABC, abstractmethod
import json

#interface superclass
class IStock(Interface):
    @abstractmethod
    def __str__():
        """Return a string representation of the stock."""

#implementation of interface
@implementer(IStock)
#superclass for stock details
class Stock(ABC):
    def __init__(self, symbol, quantity, price):
        self.symbol = symbol
        self.quantity = quantity
        self.price = price

    def __str__(self):
        return (
            f"Stock(symbol={self.symbol}, quantity={self.quantity}, price={self.price})"
        )

#extension of interface for analyzer
class IStockAnalyzer(Interface):
    @abstractmethod
    def calculate_value(stock):
        """Calculate the value of a stock."""

    @abstractmethod
    def analyze_stock(stock):
        """Analyze a stock."""


@implementer(IStockAnalyzer)
class StockAnalyzer(ABC):
    @staticmethod
    def calculate_value(stock):
        return stock.quantity * stock.price

    @staticmethod
    def analyze_stock(stock):
        value = StockAnalyzer.calculate_value(stock)
        return f"Analyzing stock {stock.symbol}: Total Value - {value}"

#extension of interface for stock reporter
class IStockReporter(Interface):
    @abstractmethod
    def generate_report(stock):
        """Generate a report for a stock."""


@implementer(IStockReporter)
class StockReporter(ABC):
    @staticmethod
    def generate_report(stock):
        value = StockAnalyzer.calculate_value(stock)
        return f"Stock Report: Symbol - {stock.symbol}, Quantity - {stock.quantity}, Price - {stock.price}, Total Value - {value}"

#extension of interface for transactions
class IStockTransaction(Interface):
    @abstractmethod
    def buy_stock(stock, quantity, price_per_unit):
        """Buy a specified quantity of stock at a given price per unit."""

    @abstractmethod
    def sell_stock(stock, quantity, price_per_unit):
        """Sell a specified quantity of stock at a given price per unit."""


@implementer(IStockTransaction)
class StockTransaction(ABC):
    @staticmethod
    def buy_stock(stock_manager, symbol, quantity, price_per_unit):
        try:
            # Check if the stock exists
            existing_stock = next(
                (s for s in stock_manager.stocks if s.symbol == symbol), None
            )

            if existing_stock:
                existing_stock.quantity += quantity
                existing_stock.price = price_per_unit
                return f"Bought {quantity} units of {existing_stock.symbol} at {price_per_unit} per unit."
            else:
                # Create a new stock if it doesn't exist
                new_stock = Stock(symbol, quantity, price_per_unit)
                stock_manager.add_stock(new_stock)
                return f"Stock {symbol} did not exist. Created and bought {quantity} units at {price_per_unit} per unit."

        except Exception as e:
            return f"Error buying stock: {e}"

    @staticmethod
    def sell_stock(stock, quantity, price_per_unit):
        try:
            if stock.quantity >= quantity:
                stock.quantity -= quantity
                stock.price = price_per_unit
                return f"Sold {quantity} units of {stock.symbol} at {price_per_unit} per unit."
            else:
                return f"Insufficient quantity of {stock.symbol} for sale."

        except Exception as e:
            return f"Error selling stock: {e}"

#extension of interface for stock manager 
class IStockManager(Interface):
    @abstractmethod
    def add_stock(stock):
        """Add a stock to the portfolio."""

    @abstractmethod
    def remove_stock(symbol):
        """Remove a stock from the portfolio."""

    @abstractmethod
    def list_stocks():
        """List all stocks in the portfolio."""

    @abstractmethod
    def get_stock_details(symbol):
        """Get details of a specific stock."""

    @abstractmethod
    def get_total_portfolio_value():
        """Get the total value of the entire stock portfolio."""

    @abstractmethod
    def clear_portfolio():
        """Clear all stocks from the portfolio."""

    @abstractmethod
    def save_portfolio(filename):
        """Save the portfolio to a file."""

    @abstractmethod
    def load_portfolio(filename):
        """Load the portfolio from a file."""

#sub-class for managing stocks
@implementer(IStockManager)
class StockManager(ABC):
    def __init__(self):
        self.stocks = []
        self.transactions = []
    #adds stock to stock watchlist
    def add_stock(self, stock):
        try:
            verifyObject(
                IStock, stock
            )  # Check if the provided object adheres to the IStock interface
            self.stocks.append(stock)
        except Exception as e:
            print(f"Error adding stock: {e}")
    #removes stock from stock lists
    def remove_stock(self, symbol):
        try:
            self.stocks = [s for s in self.stocks if s.symbol != symbol]
        except Exception as e:
            print(f"Error removing stock: {e}")

    def list_stocks(self):
        try:
            return [str(stock) for stock in self.stocks]
        except Exception as e:
            print(f"Error listing stocks: {e}")
    #returns stock details
    def get_stock_details(self, symbol):
        try:
            for stock in self.stocks:
                if stock.symbol == symbol:
                    return StockReporter.generate_report(stock)
            return f"No stock found with symbol {symbol}"
        except Exception as e:
            print(f"Error getting stock details: {e}")
    #gives total porfolio value of stocks and price
    def get_total_portfolio_value(self):
        try:
            total_value = sum(
                StockAnalyzer.calculate_value(stock) for stock in self.stocks
            )
            return f"Total Portfolio Value: {total_value}"
        except Exception as e:
            print(f"Error getting total portfolio value: {e}")
    #delete porfolio
    def clear_portfolio(self):
        try:
            self.stocks = []
            return "Portfolio cleared."
        except Exception as e:
            print(f"Error clearing portfolio: {e}")
    #save portfolio
    def save_portfolio(self, filename):
        try:
            data = {
                "stocks": [
                    {
                        "symbol": stock.symbol,
                        "quantity": stock.quantity,
                        "price": stock.price,
                    }
                    for stock in self.stocks
                ],
                "transactions": self.transactions,
            }
            with open(filename, "w") as file:
                json.dump(data, file)
            return f"Portfolio saved to {filename}."

        except Exception as e:
            print(f"Error saving portfolio: {e}")
    #loading portfolio
    def load_portfolio(self, filename):
        try:
            with open(filename, "r") as file:
                data = json.load(file)

            # Clear existing stocks before loading
            self.stocks = []

            # Populate stocks list with Stock instances
            for stock_data in data.get("stocks", []):
                symbol = stock_data.get("symbol")
                quantity = stock_data.get("quantity")
                price = stock_data.get("price")
                stock = Stock(symbol, quantity, price)
                self.stocks.append(stock)

            # Load transactions
            self.transactions = data.get("transactions", [])

            return f"Portfolio loaded from {filename}."

        except Exception as e:
            print(f"Error loading portfolio: {e}")

#final interface for user
def user_interface():
    stock_manager = StockManager()

    while True:
        print("\n=== Stock Management System ===")
        print("1. Add Stock")
        print("2. Remove Stock")
        print("3. List Stocks")
        print("4. Get Stock Details")
        print("5. Buy Stock")
        print("6. Sell Stock")
        print("7. Analyze Stock")
        print("8. Get Total Portfolio Value")
        print("9. Clear Portfolio")
        print("10. Save Portfolio")
        print("11. Load Portfolio")
        print("12. Exit")

        choice = input("Enter your choice (1-12): ")

        print()

        try:
            if choice == "1":
                symbol = input("Enter stock symbol: ")
                quantity = int(input("Enter quantity: "))
                price = float(input("Enter price per unit: "))
                stock_manager.add_stock(Stock(symbol, quantity, price))
                print(f"Stock {symbol} added.")

            elif choice == "2":
                symbol = input("Enter stock symbol to remove: ")
                stock_manager.remove_stock(symbol)
                print(f"Stock {symbol} removed.")

            elif choice == "3":
                print("Current Stocks:", stock_manager.list_stocks())

            elif choice == "4":
                symbol = input("Enter stock symbol to get details: ")
                print(stock_manager.get_stock_details(symbol))

            elif choice == "5":
                symbol = input("Enter stock symbol to buy: ")
                quantity = int(input("Enter quantity to buy: "))
                price_per_unit = float(input("Enter price per unit: "))
                transaction_result = StockTransaction.buy_stock(
                    stock_manager, symbol, quantity, price_per_unit
                )
                stock_manager.transactions.append(transaction_result)
                print(transaction_result)

            elif choice == "6":
                symbol = input("Enter stock symbol to sell: ")
                quantity = int(input("Enter quantity to sell: "))
                price_per_unit = float(input("Enter price per unit: "))
                matching_stocks = [
                    stock for stock in stock_manager.stocks if stock.symbol == symbol
                ]

                if matching_stocks:
                    transaction_result = StockTransaction.sell_stock(
                        matching_stocks[0], quantity, price_per_unit
                    )
                    stock_manager.transactions.append(transaction_result)
                    print(transaction_result)
                else:
                    print(f"No stock found with symbol {symbol}")

            elif choice == "7":
                symbol = input("Enter stock symbol to analyze: ")
                matching_stocks = [
                    stock for stock in stock_manager.stocks if stock.symbol == symbol
                ]

                if matching_stocks:
                    print(StockAnalyzer.analyze_stock(matching_stocks[0]))
                else:
                    print(f"No stock found with symbol {symbol}")

            elif choice == "8":
                print(stock_manager.get_total_portfolio_value())

            elif choice == "9":
                print(stock_manager.clear_portfolio())

            elif choice == "10":
                filename = input("Enter filename to save portfolio: ")
                print(stock_manager.save_portfolio(filename))

            elif choice == "11":
                filename = input("Enter filename to load portfolio from: ")
                print(stock_manager.load_portfolio(filename))

            elif choice == "12":
                print("Exiting the Stock Management System. Goodbye!")
                break

            else:
                print("Invalid choice. Please enter a number between 1 and 12.")

        except ValueError as ve:
            print(f"ValueError: {ve}. Please enter a valid input.")

        except Exception as e:
            print(f"Error: {e}. An unexpected error occurred.")
            break


if __name__ == "__main__":
    user_interface()



=== Stock Management System ===
1. Add Stock
2. Remove Stock
3. List Stocks
4. Get Stock Details
5. Buy Stock
6. Sell Stock
7. Analyze Stock
8. Get Total Portfolio Value
9. Clear Portfolio
10. Save Portfolio
11. Load Portfolio
12. Exit
Enter your choice (1-12): 1

Enter stock symbol: GOOGL
Enter quantity: 10
Enter price per unit: 43
Stock GOOGL added.

=== Stock Management System ===
1. Add Stock
2. Remove Stock
3. List Stocks
4. Get Stock Details
5. Buy Stock
6. Sell Stock
7. Analyze Stock
8. Get Total Portfolio Value
9. Clear Portfolio
10. Save Portfolio
11. Load Portfolio
12. Exit
Enter your choice (1-12): 1

Enter stock symbol: AMD
Enter quantity: 23
Enter price per unit: 106
Stock AMD added.

=== Stock Management System ===
1. Add Stock
2. Remove Stock
3. List Stocks
4. Get Stock Details
5. Buy Stock
6. Sell Stock
7. Analyze Stock
8. Get Total Portfolio Value
9. Clear Portfolio
10. Save Portfolio
11. Load Portfolio
12. Exit
Enter your choice (1-12): 8

Total Portfolio Value: 2868

In [3]:
#Open-Closed Principle
#Extending class stock and implementing custom functionality
class CustomStock(Stock):
    # Additional attributes and methods specific to CustomStock
    def __init__(self, symbol, quantity, price, custom_property):
        super().__init__(symbol, quantity, price)
        self.custom_property = custom_property
# Instantiate an object of CustomStock
custom_stock_instance = CustomStock(symbol="CUS", quantity=100, price=25.0, custom_property="SomeValue")
from zope.interface import Interface, implementer
from zope.interface.verify import verifyObject
from abc import ABC, abstractmethod
import json

class IStock(Interface):
    @abstractmethod
    def __str__():
        """Return a string representation of the stock."""

@implementer(IStock)
class Stock(ABC):
    def __init__(self, symbol, quantity, price):
        self.symbol = symbol
        self.quantity = quantity
        self.price = price

    def __str__(self):
        return (
            f"Stock(symbol={self.symbol}, quantity={self.quantity}, price={self.price})"
        )

class CustomStock(Stock):
    def __init__(self, symbol, quantity, price, custom_property):
        super().__init__(symbol, quantity, price)
        self.custom_property = custom_property

    def __str__(self):
        return (
            f"CustomStock(symbol={self.symbol}, quantity={self.quantity}, "
            f"price={self.price}, custom_property={self.custom_property})"
        )

def create_stock(symbol, quantity, price, custom_property=None):
    if custom_property is not None:
        return CustomStock(symbol, quantity, price, custom_property)
    else:
        return Stock(symbol, quantity, price)

class IStockAnalyzer(Interface):
    @abstractmethod
    def calculate_value(stock):
        """Calculate the value of a stock."""

    @abstractmethod
    def analyze_stock(stock):
        """Analyze a stock."""

# ... (rest of the existing code)

if __name__ == "__main__":
    user_interface()



=== Stock Management System ===
1. Add Stock
2. Remove Stock
3. List Stocks
4. Get Stock Details
5. Buy Stock
6. Sell Stock
7. Analyze Stock
8. Get Total Portfolio Value
9. Clear Portfolio
10. Save Portfolio
11. Load Portfolio
12. Exit
Enter your choice (1-12): 2

Enter stock symbol to remove: GOOGL
Stock GOOGL removed.

=== Stock Management System ===
1. Add Stock
2. Remove Stock
3. List Stocks
4. Get Stock Details
5. Buy Stock
6. Sell Stock
7. Analyze Stock
8. Get Total Portfolio Value
9. Clear Portfolio
10. Save Portfolio
11. Load Portfolio
12. Exit
Enter your choice (1-12): 12

Exiting the Stock Management System. Goodbye!
